## fNIRs storytelling order generator
The purpose of this code is to determine which and the order of the stories shown to children. 
Key points are:
* 4 types of stories
* 3 variations of story within each type


In [43]:
import numpy as np
import pandas as pd
import os
import random
print(os.getcwd())

exp_folder = '/Users/mdavistr/Desktop/lab_work/fnirs'
stim_sequences_folder = os.path.join(exp_folder,'stimulus_sequences')
stim_folder = os.path.join(exp_folder,'stimulus')
os.chdir(exp_folder)

/Users/mdavistr/Desktop/lab_work/fnirs


In [91]:
def create_directory(path):
    try:
        os.makedirs(path, exist_ok=True)
        print(f"Directory created or already exists: {path}")
    except Exception as e:
        print(f"Error creating directory {path}: {e}")

def extract_base_title(option):
    base_name = option.replace('.mp4', '')  # Remove the .mp4 extension
    return base_name.split('_')[-1] 
    


create_directory(stim_sequences_folder)

# Book types and their options
book_types = {
    'video': ['lets-be-friends', 'mouse-house', 'narwhal', 'why-do-flowers-grow'],
    'audio': ['lets-be-friends', 'mouse-house', 'narwhal', 'why-do-flowers-grow'],
    'interactive': ['lets-be-friends', 'mouse-house', 'narwhal', 'why-do-flowers-grow'],
    'Bluey': ['S01E06_The_Weekend', 'S01E07_BBQ', 'S01E29_The_Creek', 'S02E06_Bingo']}

# Book questions, (Need to be randomized order? No...)
book_questions = {
    'lets-be-friends': [
        'Who was the first creature Rosie met?', 
        'Where did Rosie and Colby go swimming?', 
        'What is Rosie trying to do at the beginning of the story?', 
        'Who ran faster when Rosie and Colby were running?', 
        'What did the friends dream about?', 
        'How did Rosie feel when she became friends with Colby?', 
        'What game(s) did Rosie and Colby play?', 
        'What animals were there in the story?'],
    'mouse-house': ['Who spun curtains for Mouse?', 
                   'Where does Mouse make their house?', 
                   'What is Mouse trying to do?', 
                   'Who thought Mouse needed more rooms?', 
                   'What did the animals do at the end of the story?', 
                   'How did the mouse feel about her new house?', 
                   'What did Goat bring?', 
                   'Who brought feathers for the bed?'],
    'narwhal': ['Who did Narwhal ask for help?', 
               'Where did Narwhal live?', 
               'When did Narwhal notice a problem?', 
               'What problem did Narwhal notice?', 
               'What did the creatures use to clean up the ocean', 
               'How did Narwhal feel about his home?', 
               'What can you always do if you don’t know how to fix a problem?', 
               'How did the town look at the end of the story?'],
    'why-do-flowers-grow': ['Who eats the flowers?', 
                           'Where do Frog and Duck meet Bee and Butterfly?', 
                           'What are the animals trying to learn?', 
                           'Who is not happy with mole’s answer?', 
                           'Why does the girl say flowers grow?', 
                           'How do flowers make the girl feel?', 
                           'What does the bee collect from the flower?', 
                           'Who do Frog and Duck have a picnic with?'],
    'Weekend': ['Who goes for a run at the start of the story?', 
                          'Where does Bingo see the walking leaf?', 
                          'What game do they play?', 
                          'Why is Bingo sad?', 
                          'Why does Bingo ask her dad to leave the door open?', 
                          'Why does the dad act mad?', 
                          'What did Bingo find while running?', 
                          'What kind of statue does the dad not want?'],
    'BBQ': ['Who likes green peppers the best?', 
                  'Where are the red plants Bingo uses for the red pepper?', 
                  'What is Bingo in charge of?', 
                  'Why does Mom stop Bingo from climbing on the bench?', 
                  'What does Bingo use for salad dressing?', 
                  'What does Bingo want to do?', 
                  'Why does Bingo turn on the hose?', 
                  'Why do the other kids think that it is raining?'],
    'Creek': [' Who gets their nails done at the creek?', 
                        'Where does Bluey want to stay instead of going to the creek?', 
                        'What does Bluey hurt when she slides down the hill?', 
                        'Which rocks are slippery?', 
                        ' What nibbles at Bluey’s toes by the waterfall?', 
                        'How does Bluey feel about going to the creek at the start of the story?', 
                        'What game do they play at the creek?', 
                        'What animal does Dad tell Bluey to watch out for?'],
    'Bingo': ['Who helps Bingo get unstuck from the railing?', 
                    'Where does Bingo get her snack from?', 
                    'Why can’t mom play with Bingo?', 
                    'What goes missing the Bingo has to find?', 
                    'Where was the missing puzzle piece?', 
                    'How does Bingo feel about having to play by herself?', 
                    'Why was Bingo hiding?', 
                    'What two fruits does Bingo choose between?']}

Directory created or already exists: /Users/mdavistr/Desktop/lab_work/fnirs/stimulus_sequences


In [93]:
# Process for each subject
for isubj in range(4):
    randomized_book_types = list(book_types.keys())
    
    # Alternate position of "interactive" based on participant index
    if isubj % 2 == 0:  # Even index (0, 2)
        randomized_book_types.remove('interactive')
        randomized_book_types.append('interactive')  # Place interactive last
    else:  # Odd index (1, 3)
        randomized_book_types.remove('interactive')
        randomized_book_types.insert(0, 'interactive')  # Place interactive first

    random.shuffle(randomized_book_types[1:])  # Shuffle the rest, keep "interactive" in position

    used_options = set()  # Track used options for both video and audio
    selected_books = {}
    
    for book_type in randomized_book_types:
        options = book_types[book_type]
        available_options = [option for option in options if option not in used_options]
        
        if available_options:
            selected_option = random.choice(available_options)
            used_options.add(selected_option)  # Track used options
            formatted_option = f"{book_type}_{selected_option.replace(' ', '_')}.mp4" if book_type in ['video', 'audio','Bluey'] else selected_option
            selected_books[book_type] = formatted_option
        else:
            print(f"Warning: No available options for {book_type}.")
    
    # Prepare data for the DataFrame
    data = {
        'Participant': [],
        'Book Type': [],
        'Selected Book Title': [],
        'Question1': [],
        'Question2': [],
        'Question3': [],
        'Question4': [],
        'Question5': [],
        'Question6': [],
        'Question7': [],
        'Question8': []
        }
  
    # Fill the DataFrame with the selected options and corresponding questions
    for book_type, option in selected_books.items():
        data['Participant'].append(f"sub-{isubj:03d}")
        data['Book Type'].append(book_type)
        data['Selected Book Title'].append(option)

        base_title = extract_base_title(option) 
        print(f"Extracted base title: {base_title}")
        
        # Fetch the questions using the base title
        questions = book_questions.get(base_title)
        

        if questions:  # Ensure questions exist for the base title
            # Append questions to respective columns
            for i in range(8):
                data[f'Question{i + 1}'].append(questions[i])
        else:
            print(f"Warning: No questions available for {base_title}")
            # Handle missing questions by appending placeholders (or take another action)
            for i in range(8):
                data[f'Question{i + 1}'].append('N/A')


    
    df = pd.DataFrame(data)
    
    # Save the DataFrame to an Excel file
    fname = os.path.join(stim_sequences_folder, f'sub-{isubj:03d}_book_stim_seq.csv')
    try:
        df.to_csv(fname, index=False)
        print(f"CSV file saved to {fname}")
    except Exception as e:
        print(f"Error saving file {fname}: {e}")
    

Extracted base title: mouse-house
Extracted base title: narwhal
Extracted base title: Bingo
Extracted base title: why-do-flowers-grow
CSV file saved to /Users/mdavistr/Desktop/lab_work/fnirs/stimulus_sequences/sub-000_book_stim_seq.csv
Extracted base title: lets-be-friends
Extracted base title: narwhal
Extracted base title: why-do-flowers-grow
Extracted base title: Creek
CSV file saved to /Users/mdavistr/Desktop/lab_work/fnirs/stimulus_sequences/sub-001_book_stim_seq.csv
Extracted base title: narwhal
Extracted base title: why-do-flowers-grow
Extracted base title: Bingo
Extracted base title: lets-be-friends
CSV file saved to /Users/mdavistr/Desktop/lab_work/fnirs/stimulus_sequences/sub-002_book_stim_seq.csv
Extracted base title: narwhal
Extracted base title: why-do-flowers-grow
Extracted base title: mouse-house
Extracted base title: BBQ
CSV file saved to /Users/mdavistr/Desktop/lab_work/fnirs/stimulus_sequences/sub-003_book_stim_seq.csv


In [63]:
for isubj in range(4):
    sequence_book = []
    sequence_audio = []
    sequence_video = []
    sequence_inter = []
    sequence_control = []

    for ibook in book_types:
        sequence_

    df = pd.DataFrame
    

NameError: name 'sequence_' is not defined